# Analysis
1 - cloudiness does not appear to be impacted by location as it seems sporadic
2 - since its still early spring in the northern hemisphere, temps are higher below and near the equator
3 - windspeed is more prominent in the northern hemisphere  however it cant be determined if thats typical or if its just a matter of the current season

In [ ]:
import random
import requests
import pandas as pd
import numpy as np
from mapconfig import api_key
from citipy import citipy
import matplotlib.pyplot as plt
import datetime as dt

# Generate Cities


lat =[]
long =[]

for i in range(3000):
    lat.append(np.random.uniform(-180, 180))
    long.append(np.random.uniform(-90,90))


weather_dicts = {'lat': lat,'long': long}
weather_data = pd.DataFrame(weather_dicts)
weather_data.head()

weather_data['closest_city'] = ""
weather_data['country'] = ""
weather_data.head()


for index, row in weather_data.iterrows():
    lat = row['lat']
    long = row['long']
    weather_data.set_value(index, 'closest_city', citipy.nearest_city(lat, long).city_name)
    weather_data.set_value(index, 'country', citipy.nearest_city(lat, long).country_code)
    
weather_data.head()
updated_df = weather_data.drop_duplicates(['closest_city'])

sample = updated_df.sample(500)
sample_city = sample["closest_city"]

# API Calls

url = "http://api.openweathermap.org/data/2.5/weather?"

params = {'appid': api_key,
          'q': '',
          'units': 'imperial'}

weather_data = []
cities = []

counter = 1
for city in sample_city:
    # Get weather data
    try:
        city1 = city.replace(" ", "%20")
        params['q'] = city1
        url_used = url + '?appid=' + api_key + '&units=' + 'imperial' '&q=' + city1
        response = requests.get(url, params=params).json()
        weather_data.append(response)
        print("Processing weather info for city " + str(counter) + ": " + city + '\n' + url_used)
        counter += 1
    except KeyError:
        "Not available"
    except AttributeError:
        "Ignore"
        
print("-----------------------------")
print("Counter completed")
print("-----------------------------")

lat_data = []
long_data = []
cloud = []
max_temp = []
humid = []
wind = []

for data in weather_data:
    try:
        lat_data.append(data['coord']['lat'])
        long_data.append(data['coord']['lon'])
        cloud.append(data['clouds']['all'])
        max_temp.append(data['main']['temp_max'])
        humid.append(data['main']['humidity'])
        wind.append(data['wind']['speed'])
        cities.append(data['name'])

    except KeyError:
        "Not available"
        
        
weather_dict = {'City': cities, 'Lat': lat_data, 'Long': long_data, 
                'Cloudiness': cloud, 'Max Temp': max_temp, 'Humidity': humid,
                'Wind Speed': wind}
weather_data_df = pd.DataFrame(weather_dict)
print(weather_data_df.head())
len(weather_data_df)

weather_data_df.count(axis = 0)

weather_data_df.to_csv("WeatherPy.csv")

# Lat vs Max Temp


date = dt.date.today
date = dt.datetime.today().strftime("%m/%d/%Y")

plt.figure(figsize=(10,5))
plt.scatter(weather_data_df['Lat'], weather_data_df['Max Temp'], edgecolor = "black", marker='o')
plt.title(date +': City Latitude vs. Max Temp')
plt.ylabel('Temperature (F)')
plt.xlabel('Latitude')
plt.grid(True)
plt.xlim(-100, 120)
plt.ylim(0, 100)

plt.savefig('maxtemp.png')

plt.style.use("seaborn")
plt.show()

# Lat vs Humidity

date = dt.date.today
date = dt.datetime.today().strftime("%m/%d/%Y")

plt.figure(figsize=(10,5))
plt.scatter(weather_data_df['Lat'], weather_data_df['Humidity'], edgecolor = "black", marker='o')

plt.title(date +': City Latitude vs. Humidity')
plt.ylabel('Humiditiy (%)')
plt.xlabel('Latitude')
plt.grid(True)
plt.xlim(-100, 120)
plt.ylim(0, 120)

plt.savefig('humidity.png')

plt.style.use("seaborn")
plt.show()

# Lat vs cloudiness

date = dt.date.today
date = dt.datetime.today().strftime("%m/%d/%Y")

plt.figure(figsize=(10,5))
plt.scatter(weather_data_df['Lat'], weather_data_df['Cloudiness'], edgecolor = "black", marker='o')

plt.title(date +': City Latitude vs. Cloudiness')
plt.ylabel('Cloudiness (%)')
plt.xlabel('Latitude')
plt.grid(True)
plt.xlim(-100, 100)
plt.ylim(-20, 120)

plt.savefig('cloudiness.png')

plt.style.use("seaborn")
plt.show()

# Lat vs winds


date = dt.date.today
date = dt.datetime.today().strftime("%m/%d/%Y")

plt.figure(figsize=(10,5))
plt.scatter(weather_data_df['Lat'], weather_data_df['Wind Speed'], edgecolor = "black", marker='o')

plt.title(date +': City Latitude vs. Wind Speed')
plt.ylabel('Wind Speed (mph)')
plt.xlabel('Latitude')
plt.grid(True)
plt.xlim(-100, 100)
plt.ylim(-5, 40)

plt.savefig('winds.png')

plt.style.use("seaborn")
plt.show()